In [1]:
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
import tkinter as tk
from pysrc.simplertplot.plotwidget import RTPlotWidget
from pysrc.simplertplot import queues
from hello.hello3 import HelloApp, TrueError
from time import time, sleep
import numpy as np
from matplotlib.ticker import NullFormatter, NullLocator, FormatStrFormatter

In [3]:
#ip = '192.168.1.85'
ip = '71.189.82.196:88'

In [4]:
interval = 1000
minutes = 60
max_pts=minutes*60/(interval/1000)
max_pts=int(max_pts)
max_pts

3600

In [15]:
class MyPlotWidget(RTPlotWidget):
    def __init__(self, *args, **kw):
        super().__init__(*args, **kw)
        self.y = None
        self.x = None
        #self._special()

    def _update_data(self):
        self.line.set_data(self.x_data.get(), self.y_data.get())
        #self.line2.set_data(self.x_data.get(), self.y_data2.get())
        self.subplot.relim()
        self.subplot.autoscale_view(True, True, True)
        #self.ax2.relim()
        #self.ax2.autoscale_view(True,True,True)
        lower, upper = self.subplot.get_ybound()
        self.subplot.set_ylim(lower-0.2, upper+0.2, True, None)
        #lower, upper = self.ax2.get_ybound()
        #self.ax2.set_ylim(lower-1, upper+1, True, None)
        self._update()
        
    def _special(self):
        self.ax2 = self.subplot.twinx()
        self.y_data2 = queues.RingBuffer(self.max_pts)
        self.line2, = self.ax2.plot(self.x_data.get(), self.y_data2.get(), 'g-')
        self.yaxis2 = self.ax2.yaxis
        self.yaxis2.set_major_formatter(FormatStrFormatter("%.2f"))

    def extend_xy(self, x, y, y2):
        
        if self.y is None:
            self.x = x
            self.y = y
        else:
            self.y += (x-self.x)*(y-self.y)*0.1
            y = self.y
        self.x_data.extend(x)
        self.y_data.extend(y)
        
        #self.y_data2.extend(y2)
        self._update_data()
        
    def legend(self, text1):
        self.line.set_label(text1)
        #self.line2.set_label(text2)
#         l1.set_text(text1)
#         l2.set_text(text2)
        h1, l1 = self.subplot.get_legend_handles_labels()
        #h2, l2 = self.ax2.get_legend_handles_labels()
        #h1.extend(h2)
        #l1.extend(l2)
        self.subplot.legend(h1, l1)
        #self.subplot.legend(loc='upper right')
        #self.ax2.legend(loc='lower right')
        
    def reset(self):
        self.x_data.clear()
        self.y_data.clear()
        #self.y_data2.clear()
        self._update_data()
        self.x = self.y = None

    def add_xy(self, x, y):
        if self.y is None:
            self.x = x
            self.y = y
        else:
            self.y += (x-self.x)*(y-self.y)*0.1
            #y = self.y
            self.y = y
            self.x = x
        self.x_data.put(x)
        self.y_data.put(y)
        self._update_data()

In [16]:
def timer():
    start = time()
    while True:
        current = time()
        yield current - start
get_time = timer().__next__

In [17]:
g_stop_tasks = False
g_after = None
def tk_task(tk, interval, f):
    global g_after, g_stop_tasks
    def fn(*args, **kw):
        global g_after
        try:
            return f(*args, **kw)
        finally:
            if not g_stop_tasks:
                g_after = tk.after(interval, fn)
    g_after = tk.after(interval, fn)

In [18]:
controller = 'level'

In [19]:
import threading
class Model():
    def __init__(self, app, ctrl, interval=0.1):
        self.mv = app.getMainValues()  # prepopulate
        self.time = get_time()
        self.lock = threading.RLock()
        self.running = threading.Event()
        self.thread = None
        self.app = app
        self.interval = interval
        self.ctrl = ctrl
    def getvalues(self):
        with self.lock:
            pv = self.mv[self.ctrl]['pv']
            t = self.time
        return t, pv
    def getMainValues(self):
        return self.mv
    def run_update(self):
        self.running.set()
        self.thread = threading.Thread(None, self._run_update, daemon=True)
        self.thread.start()
    def _run_update(self):
        while self.running.is_set():
            try:
                mv = self.app.getMainValues()
            except Exception:
                self.app.reconnect()
                continue
            with self.lock:
                self.mv = mv
                self.time = get_time()
            sleep(self.interval)
    def stop(self):
        self.running.clear()
        self.thread.join(timeout=10)
        if self.thread.is_alive():
            raise RuntimeError("Failed to stop thread")
    def reset(self):
        self.time = get_time()
        self.stop()
        self.run_update()

h = HelloApp(ip)
m = Model(h, controller, 0.5)
m.run_update()

def update():
    t, pv= m.getvalues()
    p.add_xy(t, pv)
    update_labels(pv)

In [20]:
def printdir(o):
    for a in dir(o):
        if not a.startswith("_"):
            print(a, getattr(o, a))

In [21]:
def conf2(ev):
    if ev.widget == root:
        p.tkcanvas.config(height=ev.height, width=ev.width-rbtn.winfo_width())

In [22]:
root = tk.Tk()
root.withdraw()

p = MyPlotWidget(root, max_pts=max_pts)
p.tkcanvas.config(height=400, width=600)
max_pts


3600

In [23]:
class ILabel(tk.Label):
    def __init__(self, master, name, value=0, fmt="%.2f"):
        super().__init__(master)
        self.fmt = fmt
        self.name = name
        self.set_value(value)
        
    def set_value(self, val):
        val = self.fmt % val
        self.config(text="%s: %s" % (self.name, val))
   
labels = {}
def mk_label(name, val, row):
    l = ILabel(root, name, val)
    labels[name] = l
    l.grid(row=row, column=1)
    
mk_label("PV", 0, 1)
p.legend("PV")
    
def update_labels(pv):
    labels['PV'].set_value(pv)

    

In [24]:
p.grid(rowspan=20, column=0, row=0)    
def reset():
    p.reset()
    global get_time
    get_time = timer().__next__
    m.reset()
rbtn = tk.Button(root, text="Reset", command=reset)
rbtn.grid(column=1, row=0, sticky="ew")
root.bind("<Configure>", conf2)
p.set_xaxis_text("Time (s)")
p.set_yaxis_text("%s PV" % controller.title())
def on_closing():
    from tkinter import messagebox
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        global g_stop_tasks, g_after
        g_stop_tasks = True
        root.after_cancel(g_after)
        root.quit()
        root.destroy()
p.subplot.grid()
root.protocol("WM_DELETE_WINDOW", on_closing)
root.wm_deiconify()
tk_task(root, interval, update)
try:
    root.mainloop()
finally:
    m.stop()
